This script is made to update the defensewiki pages, as we have country names not saved and also we have duplicates.
We want to add as metadata the country in all the defense wiki pages.
Update also chunks and collection of chromadb
Some countries have duplicated pages, we have hashes that are not unique

In [1]:
import os
os.chdir("/Users/dianaavalos/PycharmProjects/InternationalBridgesToJustice")
print(os.getcwd())

/Users/dianaavalos/PycharmProjects/InternationalBridgesToJustice


In [16]:
import json
from src.scraping_functions import matching_country_name
from src.config import path_folder_defense_wiki, file_country_names
from src.countries_dict import title_to_country, substring_to_country
from src.file_manager import save_file
from collections import defaultdict, Counter
import pandas as pd
import chromadb
import numpy as np
from src.config import path_chromadb, collection_name

load defensewiki to update countries

In [3]:
json_filename=f"{path_folder_defense_wiki}/defensewiki_no_content.json"
with open(json_filename, "r", encoding="utf-8") as f:
    dict_defensewiki = json.load(f)

In [4]:
jsonl_filename=f"{path_folder_defense_wiki}/defensewiki_all.jsonl"
with open(jsonl_filename, "r", encoding="utf-8") as f:
    dict_defensewiki_all = [json.loads(line) for line in f]

In [5]:
print(type(dict_defensewiki))
print(type(dict_defensewiki_all))

<class 'dict'>
<class 'list'>


Notice that dict_defensewiki_all is a list of dict, and dict_defensewiki is a dict of dict.
What is best? dict of dict more practical to look up data[key], does not allow duplicates. if you use it like a table, or export to pandas, might be easier list of dict?

remove first key:
'https://defensewiki.ibj.org/index.php?title=Special:MostRevisions&limit=1300&offset=0'

In [6]:
first_key = next(iter(dict_defensewiki))
first_key

'https://defensewiki.ibj.org/index.php?title=Special:MostRevisions&limit=1300&offset=0'

In [7]:
dict_defensewiki.pop(first_key)
first_key = next(iter(dict_defensewiki))
first_key

'https://defensewiki.ibj.org/index.php?title=Criminal_Justice_Systems_Around_the_World'

In [8]:
first_key = next(iter(dict_defensewiki_all))

Update all the countries for each

In [9]:
with open(file_country_names, "r", encoding="utf-8") as f:
    list_country_names = f.read().splitlines()

In [12]:
for d in dict_defensewiki.values():
    d["country"] = matching_country_name(list_country_names, d["title"], title_to_country, substring_to_country)
for d in dict_defensewiki_all[1:]:
    d["country"] = matching_country_name(list_country_names, d["title"], title_to_country, substring_to_country)

Debug

In [13]:
matching_country_name(list_country_names, "china criminal defense manual - motions", title_to_country, substring_to_country)

'China'

Check all the titles that have no country, in case we missed one

In [14]:
for key, val in list(dict_defensewiki.items()):
    if val["country"] == "":
        print(val["title"])

criminal justice systems around the world
representing victims of torture
cross-examination
evidence
defenses
crimes
international law
nullity of procedure
rights of the accused
legal aid systems and supporting ngos around the world
crimes against humanity
right to counsel
sample cross-examination transcripts
pre-trial detention
international criminal court
confessions
false confessions - admissions
sistemas de justicia penal
eyewitness misidentification
eyewitness misidentification-fr
42 u.s.c.  1983
rights of the accused-es
theory of the case
arrest
exclusionary rule
justiciability doctrine
right to silence
torture
client interviews
identity checks
showups, lineups, and photo arrays
search and seizure
actus reus (voluntary act)
right to habeas corpus
double jeopardy
ethics and professional responsibility
mens rea (culpable mental state)
criminal defense resources on the web
closing statements
false confessions - admissions-fr
rome statute of the international criminal court
appeals
c

In [24]:
save_file(filename=f"{path_folder_defense_wiki}/defensewiki_all.jsonl", content=dict_defensewiki_all, file_type="jsonl")

Error saving file data/processed/defensewiki.ibj.org/defensewiki_all.jsonl: list indices must be integers or slices, not dict


When content is a list, record is each individual dict.

Accessing content[record] assumes record is an index or key, which causes the error:
TypeError: list indices must be integers or slices, not dict.

            jsonl_file.write(json.dumps(content[record]) + "\n")

changed to:

            jsonl_file.write(json.dumps(record, ensure_ascii=False) + "\n")


In [25]:
filename=f"{path_folder_defense_wiki}/defensewiki_all.jsonl"
with open(jsonl_filename, "w", encoding="utf-8") as jsonl_file:
    for record in dict_defensewiki_all:
        jsonl_file.write(json.dumps(record, ensure_ascii=False) + "\n")

In [26]:
save_file(filename=f"{path_folder_defense_wiki}/defensewiki_no_content.json", content=dict_defensewiki, file_type="json")

File saved successfully: data/processed/defensewiki.ibj.org/defensewiki_no_content.json


Update the chunks with country names

In [64]:
# load the chunks and add countries
path1 = "data/processed/defensewiki.ibj.org/chunks_1.jsonl"
chunks = []
for path in [path1]:
    with open(f"{path}", "r", encoding="utf-8") as jsonl_file:
        lines = jsonl_file.readlines()  # Read all lines once
        print(f"Number of lines in the jsonl file {path}: {len(lines)}")
        for line in lines:
            chunks.append(json.loads(line))
        for chunk in chunks:
            matches = [
                d["country"]
                for d in dict_defensewiki_all[1:]
                if d["title"].strip().lower()  == chunk["metadata"]["title"].strip().lower()
            ]
            chunk["metadata"]["country"] = matches[0] if matches else ""
    print(f"Total number of chunks: {len(chunks)}")

# check we have countries
seen_countries = set([chunk['metadata']['country'] for chunk in chunks])

Number of lines in the jsonl file data/processed/defensewiki.ibj.org/chunks_1.jsonl: 10941
Total number of chunks: 10941


Check it worked

In [65]:
dict_defensewiki_all[2].keys()
print(dict_defensewiki_all[2]["title"])
print(dict_defensewiki_all[2]["country"])

singapore
Singapore


In [66]:
print(chunks[10]["metadata"]["title"])
print(chunks[10]["metadata"]["country"])

Singapore
Singapore


In [70]:
with open(
        "data/processed/defensewiki.ibj.org/chunks_1.json", "w", encoding="utf-8"
) as file:
    json.dump(chunks, file, indent=4)  # Save JSON content

with open(
        "data/processed/defensewiki.ibj.org/chunks_1.jsonl", "w", encoding="utf-8"
) as jsonl_file:
    for record in chunks:
        jsonl_file.write(json.dumps(record) + "\n")


In [72]:
[d for d in chunks[1:] if d["metadata"]["country"] == "Burundi"][1:2]
# [d for d in dict_defensewiki_all[1:] if d["country"] == "Burundi"]

[{'title': 'e6cb3b89f3230472b811c58b6503c085bb9d5a5c1056dd0b1a63754a789ea75c',
  'content': 'Background of Burundi\n---------------------\n\nBurundi is an African country located in the Great Lakes region and is one of the six member states of the East African Community (EAC). With a total area of 27,834 km, Burundi is one of the smallest countries in Africa. Landlocked, Burundi shares borders with Tanzania to the East, the Democratic Republic of the Congo to the West, and Rwanda to the North. Burundi contains Lake Tanganyika and other smaller lakes that dot the country. The overall population density compared to the area of the land (not including the lakes) is 435 inhabitants per km2 [[1]](#cite_note-1), which is concentrated mainly in the provinces of Gitega, Kayanza, Ngozi, Kirundo, and Muyinga, making Burundi one of the most populous African countries. Burundi officially recognizes three ethnic groups: Hutu (85% of the population), Tutsi (14% of the population), and Twa (1% of the

Change embeddings and remove duplicates 

In [ ]:
input_data = "data/processed/defensewiki.ibj.org/chunks_1.json"
with open(input_data, "r", encoding="utf-8") as json_file:
    chunks = json.load(json_file)

# get duplicated hashes
titles = [chunk["title"] for chunk in chunks]
len(titles) == len(np.unique(titles))
# len(np.unique(titles)) 10605
# len(titles) 10941
duplicated_hash = [item for item, count in Counter(titles).items() if count > 1]
# get all duplicated chunks
duplicated_chunks = [d for d in chunks if d["title"] in duplicated_hash]
len(duplicated_chunks)
# extract pages names
duplicated_pages_data = [
    [
        d["title"],
        d["metadata"]["title"],
        d["metadata"]["link"],
        d["metadata"]["title_bis"],
    ]
    for d in duplicated_chunks
]
df_duplicates = pd.DataFrame(
    duplicated_pages_data, columns=["Hash Title", "Page Title", "Link", "Title Bis"]
)

output_file = "data/interim/duplicated_hashes_and_pages.csv"
df_duplicates.to_csv(output_file, sep="\t", index=False)

# if duplicate, only keep first hash ------------------------

seen = set() # unordered and unique
unique_chunks = []

for chunk in chunks:
    title = chunk["title"]
    if title not in seen:
        seen.add(title)
        unique_chunks.append(chunk)

# create a new file with unique_chunks ------------------------

path = "data/processed/defensewiki.ibj.org"

# JSON Lines (one dict per line)
with open(f"{path}/unique_chunks.jsonl", "w", encoding="utf-8") as jsonl_file:
    for chunk in unique_chunks:
        jsonl_file.write(json.dumps(chunk) + "\n")

# JSON (single array of dicts)
with open(f"{path}/unique_chunks.json", "w", encoding="utf-8") as json_file:
    json.dump(unique_chunks, json_file, ensure_ascii=False, indent=2)


# add country (and these changes) to the collection -----------------------------------
# In ChromaDB, embeddings and metadata are separate
# you don’t have to re-embed your text just to update or add metadata


client = chromadb.PersistentClient(path=path_chromadb)
collection = client.get_or_create_collection(name=collection_name)

# Group chunks by title
title_to_chunk = defaultdict(list)
for chunk in chunks:
    title_to_chunk[chunk["title"]].append(chunk)

unique_chunks = [chunks[0] for chunks in title_to_chunk.values()]

cleaned_chunks = []
for chunk in unique_chunks:
    country = chunk["metadata"].get("country", "Unknown")
    chunk["metadata"]["country"] = str(country) if country is not "" else "Unknown"
    cleaned_chunks.append(chunk)

collection.upsert(
    ids=[chunk["title"] for chunk in unique_chunks],
    documents=[chunk["content"] for chunk in cleaned_chunks],
    metadatas=[{"country": chunk["metadata"]["country"]} for chunk in cleaned_chunks],
)

valid_chunks = [
    chunk
    for chunk in cleaned_chunks
    if isinstance(chunk["content"], str) and chunk["content"].strip() != ""
]
len(valid_chunks)
len(cleaned_chunks)


# # Get all the IDs from the collection
# documents = collection.get()
# existing_ids = [doc["id"] for doc in documents["documents"]]
# len(np.unique(documents['ids'])) == len(documents['ids'])
